In [1]:
import requests
from requests import get
from requests.exceptions import RequestException
from user_agent import generate_user_agent, generate_navigator
from contextlib import closing
from bs4 import BeautifulSoup
from lxml import html
import re
import csv
import pandas as pd
import os
from pathlib import Path

headers = generate_navigator()
print(headers)

{'os_id': 'win', 'navigator_id': 'chrome', 'platform': 'Windows NT 5.1', 'oscpu': 'Windows NT 5.1', 'build_version': '53.0.2794.98', 'build_id': None, 'app_version': '5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2794.98 Safari/537.36', 'app_name': 'Netscape', 'app_code_name': 'Mozilla', 'product': 'Gecko', 'product_sub': '20030107', 'vendor': 'Google Inc.', 'vendor_sub': '', 'user_agent': 'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2794.98 Safari/537.36'}


In [2]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True, timeout=1, headers=headers)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None
    
def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [3]:
def get_all_links(soup, homepage):
    pages = []
    for item in soup.find_all('a'):
        page = item.get('href')
        if page != None:
            if page[:3] == '../':
                page = homepage + page[3:]
        pages.append(page)
    return list(set(pages))

In [4]:
def get_url_info(url):
    try:
        homepage = 'https://' + url
        raw_html = simple_get(homepage)
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
    
    if raw_html != None:
        soup = BeautifulSoup(raw_html, 'html.parser')
    else:
        return None
    
    try:
        title = soup.title.get_text()
    except AttributeError:
        title = None
        
    if '|' in title:  # TODO is this the case for all?
        title = title.split('|')[0]
    text = soup.get_text()
    
    pages = get_all_links(soup, homepage)
    
    return pages, text, title

In [5]:
def check_string_in_list(string, arr):
    arr = list(set(arr))
    
    for item in arr:
        if string in item:
            return item

In [6]:
def get_company_sites(str_list, pages):
    sites = []
    
    for item in str_list:
        # Problem iterating 3 times TODO
        site = check_string_in_list(item, pages)
        sites.append(site)
    
    return sites

In [7]:
def get_company_urls(csv):
    company_urls = []
    
    data = pd.read_csv(csv, sep=';', encoding='latin1', names=['Domain', 'Section', 'Code'])
    
    for index, row in data.iterrows():
        company_urls.append(row['Domain'])
        
    return company_urls

In [8]:
company_csv = os.path.join(Path(os.getcwd()).parent.parent, 'deeptech-ai-data', '20181018 Domain+WZ2008 Hackathon INOBAS.csv')

In [9]:
print(company_csv)

/home/tony/deeptech-ai-data/20181018 Domain+WZ2008 Hackathon INOBAS.csv


In [10]:
company_urls = get_company_urls(company_csv)[1:]  # skipping first item header

In [11]:
def get_c_title(text):
    end_c = re.search("©", text).end()
    start_nl = re.search("\n", text[end_c:]).start()
    title = text[end_c+1:end_c+start_nl]
    
    return title

In [12]:
def run(company_urls):
    infos = {}
    
    count = 0
    fails = 0
    for url in company_urls:
        if count % 100 == 0:
            successes = count - fails
            print("Count:", count, "\t", "Successes:", successes, "Fails:", fails, "URL:", url)
        try:
            pages, text, title = get_url_info(url)
#             pages, title = get_url_info(url)
            try:
                c_title = get_c_title(text)
            except AttributeError:
                c_title = "PAGE TITLE: " + title
            specific_pages = get_company_sites(['facebook', 'impressum'], pages)
#             infos[url] = [title, specific_pages, None] # TODO 
            infos[url] = [c_title, specific_pages, None]
            count += 1
        except TypeError:
            fails += 1
            count += 1
            infos[url] = [None, None, None]
            continue
        
    return infos

In [13]:
infos = run(company_urls)

Count: 0 	 Successes: 0 Fails: 0 URL: www.hoberg-driesch.de
Error during requests to https://www.hgdf.de : HTTPSConnectionPool(host='www.hgdf.de', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f8685877160>, 'Connection to www.hgdf.de timed out. (connect timeout=1)'))
Error during requests to https://www.gasag.de : HTTPSConnectionPool(host='www.gasag.de', port=443): Read timed out. (read timeout=1)
Error during requests to https://www.wahler.de : HTTPSConnectionPool(host='www.wahler.de', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f86860df5f8>: Failed to establish a new connection: [Errno 111] Connection refused',))
Error during requests to https://www.msa-auer.de : HTTPSConnectionPool(host='www.msa-auer.de', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPS

Error during requests to https://www.eduard-hueck.de : HTTPSConnectionPool(host='www.eduard-hueck.de', port=443): Max retries exceeded with url: / (Caused by SSLError(CertificateError("hostname 'www.eduard-hueck.de' doesn't match either of '*.hueck.com', 'hueck.com'",),))
Count: 100 	 Successes: 38 Fails: 62 URL: www.tui-business-services.de
Error during requests to https://www.tui-business-services.de : HTTPSConnectionPool(host='www.tui-business-services.de', port=443): Read timed out. (read timeout=1)
Error during requests to https://www.roche.de : HTTPSConnectionPool(host='www.roche.de', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f868528dbe0>, 'Connection to www.roche.de timed out. (connect timeout=1)'))
Error during requests to https://www.bkk-basell.de : HTTPSConnectionPool(host='www.bkk-basell.de', port=443): Max retries exceeded with url: / (Caused by SSLError(CertificateError("hostname 'www

Error during requests to https://www.otto-junker.de : HTTPSConnectionPool(host='www.otto-junker.de', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:833)'),))
Count: 200 	 Successes: 78 Fails: 122 URL: www.kreiskliniken-traunstein-trostberg.de
Error during requests to https://www.kreiskliniken-traunstein-trostberg.de : HTTPSConnectionPool(host='www.kreiskliniken-traunstein-trostberg.de', port=443): Max retries exceeded with url: / (Caused by SSLError(CertificateError("hostname 'www.kreiskliniken-traunstein-trostberg.de' doesn't match 'www.backorder.de'",),))
Error during requests to https://www.wieland-electric.de : HTTPSConnectionPool(host='www.wieland-electric.de', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f86850784e0>: Failed to establish a new connection: [Errno 111] Connection refused',))
E

Error during requests to https://www.pgp.de : HTTPSConnectionPool(host='www.pgp.de', port=443): Max retries exceeded with url: / (Caused by SSLError(CertificateError("hostname 'www.pgp.de' doesn't match either of 'symantec.com', 'norton.com', 'account.norton.com', 'careers.symantec.com', 'customercare.symantec.com', 'de.norton.mobi', 'downloads.guardianedge.com', 'emea.symantec.com', 'eu.store.pgp.com', 'jobs.symantec.com', 'mostdangeroustown.com', 'mynortonaccount.com', 'na.store.pgp.com', 'nortonaccount.com', 'nortonlearninghub.com', 'nukona.com', 'row.store.pgp.com', 'ssl.symantec.com', 'store.pgp.com', 'uk.store.pgp.com', 'www.account.norton.com', 'www.emea.symantec.com', 'www.mostdangeroustown.com', 'www.nortonaccount.com', 'www.nortonlearninghub.com', 'www.nukona.com', 'www.pgp.com', 'www.ssl.symantec.com', 'www.symantec.co.jp', 'www.symantec.co.uk', 'www.symantec.fr', 'www.symantec.de', 'www.symantec.it', 'www.symantec.com.au', 'www.symantec.com.br', 'www.symantec.mx', 'www.syma

Error during requests to https://www.noelke.de : HTTPSConnectionPool(host='www.noelke.de', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f8684a814e0>, 'Connection to www.noelke.de timed out. (connect timeout=1)'))
Error during requests to https://www.krone.de : HTTPSConnectionPool(host='www.krone.de', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f8684a81b00>: Failed to establish a new connection: [Errno 111] Connection refused',))
Error during requests to https://www.gmh-holding.de : HTTPSConnectionPool(host='www.gmh-holding.de', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)'),))
Error during requests to https://www.bonifatius-lingen.de : HTTPSConnectionPool(host='www.bonifatius-lingen.de', port=443): Max retries e

...

Error during requests to https://www.hello-people.de : HTTPSConnectionPool(host='www.hello-people.de', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:833)'),))
Error during requests to https://www.drstern.de : HTTPSConnectionPool(host='www.drstern.de', port=443): Max retries exceeded with url: / (Caused by SSLError(CertificateError("hostname 'www.drstern.de' doesn't match either of '*.kasserver.com', 'kasserver.com'",),))
Error during requests to https://www.waldhotel-stuttgart.de : HTTPSConnectionPool(host='www.waldhotel-stuttgart.de', port=443): Read timed out. (read timeout=1)
Error during requests to https://www.roegelein.de : HTTPSConnectionPool(host='www.roegelein.de', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f86853f0e10>: Failed to establish a new connection: [Errno 113] No route to host',))
Error during request

Error during requests to https://www.bergmeir-dienstleistungen.de : HTTPSConnectionPool(host='www.bergmeir-dienstleistungen.de', port=443): Read timed out. (read timeout=1)
Error during requests to https://www.cellent-mittelstandsberatung.de : HTTPSConnectionPool(host='www.cellent-mittelstandsberatung.de', port=443): Read timed out. (read timeout=1)
Error during requests to https://www.moog.de : HTTPSConnectionPool(host='www.moog.de', port=443): Max retries exceeded with url: / (Caused by SSLError(CertificateError("hostname 'www.moog.de' doesn't match either of '*.moog.com', '*.live.moog.com', '*.stage.moog.com', 'broadreachengineering.com', 'csaengineering.com', 'csaengr.com', 'emactuator.com', 'flo-tork.com', 'moog-csa.com', 'moog-csaengineering.com', 'moog-memmingen.com', 'moog.co.in', 'moog.com', 'moog.com.ru', 'moog.com.sg', 'moog.dk', 'moog.es', 'moog.fr', 'moogcomponentsgroup.com', 'moogcsa.com', 'moogdenmark.dk', 'moogfernau.com', 'moogglobalsuport.com', 'moogglobalsupport.com'

Error during requests to https://www.mangold-personalpartner.de : HTTPSConnectionPool(host='www.mangold-personalpartner.de', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:833)'),))
Error during requests to https://www.pvt.de : HTTPSConnectionPool(host='www.pvt.de', port=443): Max retries exceeded with url: / (Caused by SSLError(CertificateError("hostname 'www.pvt.de' doesn't match either of 'dialog-62-test.roche.com', 'author-eu-Stage2.roche.com', 'author-eu-SuppDev2.roche.com', 'author-eu-Train.roche.com', 'author-eu-dev.roche.com', 'author-eu-owpdev.roche.com', 'author-eu-prod.roche.com', 'author-eu-qa.roche.com', 'author-eu-stage.roche.com', 'campaigns-test.roche.com', 'campaigns.roche.com', 'cssportal.roche.com', 'diagnostics-dev.roche.com', 'diagnostics-owp-dev.roche.com', 'diagnostics-qa.roche.com', 'diagnostics-test.roche.com', 'diagnostics.roche.com', 'dialog-62.roche.com', 'dialog-dev.roche.com', '

Error during requests to https://www.gmelich.de : HTTPSConnectionPool(host='www.gmelich.de', port=443): Max retries exceeded with url: / (Caused by SSLError(CertificateError("hostname 'www.gmelich.de' doesn't match either of '*.swebian.com', 'swebian.com'",),))
Error during requests to https://www.benninger-bettsysteme.de : HTTPSConnectionPool(host='www.benninger-bettsysteme.de', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f868551b908>: Failed to establish a new connection: [Errno 113] No route to host',))
Error during requests to https://www.sedia-kuechentechnik.de : HTTPSConnectionPool(host='www.sedia-kuechentechnik.de', port=443): Max retries exceeded with url: / (Caused by SSLError(CertificateError("hostname 'www.sedia-kuechentechnik.de' doesn't match either of '*.itandtel.at', 'itandtel.at'",),))
Error during requests to https://www.prima-gebaeudeservice.de : HTTPSConnectionPool(host='www.prima

KeyError: 'content-type'

In [ ]:
def convert_dict_to_csv(infos, company_urls, out_file):
#     df = pd.DataFrame(columns=['Homepage', 'Title', 'Pages', 'Text'])
    df = pd.DataFrame(columns=['Homepage', 'Title', 'Pages')
    
    for i in range(len(company_urls)):
        if company_urls[i] in infos:
            df.loc[i] = [company_urls[i], infos[company_urls[i]][0], infos[company_urls[i]][1], infos[company_urls[i]][2]]
        
    df.to_csv(out_file, sep=',', encoding='utf-8')
    return df

In [ ]:
out_file = os.path.join(Path(os.getcwd()).parent.parent, 'deeptech-ai-data', 'output.csv')

df = convert_dict_to_csv(infos, company_urls, out_file)
df